In [1]:
from neo4j import GraphDatabase

root_dir = '/Users/hp/workbench/data/json_dir'

In [2]:

class NeoExec(object):

    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()

    def exec(self, stmt):
        with self._driver.session() as session:
            result = session.write_transaction(self._exec, stmt)
            return result.values

    @staticmethod
    def _exec(tx, stmt):
        result = tx.run(stmt)
        return result


In [3]:
neo = NeoExec(uri="bolt://localhost:7687", user="neo4j", password="password")

In [4]:
def extract_quads():
    has_more = True
    pages = 10
    skip = 0
    limit = 10
    while has_more & pages>0:
        stmt: str = """match (v1:vendors)-[r1:worked_with]-(v2:vendors)-[r2:worked_with]-(v3:vendors)-[r3:worked_with]-(v4:vendors)
                where r1.claimnumber = r2.claimnumber = r3.claimnumber and id(v1) < id(v2) < id(v3) < id(v4)
                return v1, v2, v3, v4, r1, r2, r3 skip {skip} limit {limit}"""
        results = neo.exec(stmt)
        pages -= 1
        print(results)
        if results=="none":
            has_more = False
        else:
            skip += limit
            limit += limit


In [5]:
extract_quads()